# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.46,85,6,2.76,US,1724889266
1,1,waitangi,-43.9535,-176.5597,11.55,92,83,10.47,NZ,1724889267
2,2,ancud,-41.8697,-73.8203,9.76,83,3,1.91,CL,1724888872
3,3,chibougamau,49.9168,-74.3659,13.79,58,0,2.57,CA,1724888940
4,4,sao vicente,-23.9631,-46.3919,17.77,26,74,1.16,BR,1724889255


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
# longitude = city_data_df['Lng']
# latitude = city_data_df['Lat']
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = "Humidity"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,beyneu,45.3167,55.2000,21.83,29,0,2.56,KZ,1724889278
45,45,emmett,43.8735,-116.4993,24.80,12,0,3.09,US,1724889156
97,97,colonia,50.9333,6.9500,22.76,72,0,1.03,DE,1724889375
219,219,emerald,-23.5333,148.1667,23.52,67,0,1.84,AU,1724889519
263,263,saint-denis,48.9167,2.3333,22.27,67,0,2.57,FR,1724889568
304,304,viransehir,37.2353,39.7631,26.78,40,0,2.77,TR,1724889615
339,339,clearwater,27.9659,-82.8001,25.99,90,0,0.00,US,1724889591
386,386,kone,-21.0595,164.8658,26.90,41,0,4.38,NC,1724889710
465,465,remire-montjoly,4.9167,-52.2667,25.02,83,0,1.03,GF,1724889828
570,570,winnemucca,40.9730,-117.7357,26.60,16,0,4.05,US,1724889940


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,beyneu,KZ,45.3167,55.2000,29,
45,emmett,US,43.8735,-116.4993,12,
97,colonia,DE,50.9333,6.9500,72,
219,emerald,AU,-23.5333,148.1667,67,
263,saint-denis,FR,48.9167,2.3333,67,
304,viransehir,TR,37.2353,39.7631,40,
339,clearwater,US,27.9659,-82.8001,90,
386,kone,NC,-21.0595,164.8658,41,
465,remire-montjoly,GF,4.9167,-52.2667,83,
570,winnemucca,US,40.9730,-117.7357,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [64]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    #pprint(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
beyneu - nearest hotel: No hotel found
emmett - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
emerald - nearest hotel: No hotel found
saint-denis - nearest hotel: No hotel found
viransehir - nearest hotel: No hotel found
clearwater - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
winnemucca - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
10,beyneu,KZ,45.3167,55.2000,29,No hotel found
45,emmett,US,43.8735,-116.4993,12,No hotel found
97,colonia,DE,50.9333,6.9500,72,No hotel found
219,emerald,AU,-23.5333,148.1667,67,No hotel found
263,saint-denis,FR,48.9167,2.3333,67,No hotel found
304,viransehir,TR,37.2353,39.7631,40,No hotel found
339,clearwater,US,27.9659,-82.8001,90,No hotel found
386,kone,NC,-21.0595,164.8658,41,No hotel found
465,remire-montjoly,GF,4.9167,-52.2667,83,No hotel found
570,winnemucca,US,40.9730,-117.7357,16,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size=20,
    color="blue",
    hover_cols=["City", "Country", "Hotel Name"],
    frame_width=800,
    frame_height=600
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)